# Advanced Python OOP — Banking System (DDD + Patterns)

This notebook demonstrates an **advanced OOP design** for a banking system using:
- **Domain-Driven Design (DDD)** style layers (Entities, Value Objects, Services, Repositories)
- **Abstract Base Classes**, **dataclasses**, **type hints**
- **Strategy Pattern** (overdraft, interest calculation)
- **State Pattern** (account lifecycle: Active/Frozen/Closed)
- **Observer Pattern** (notifications)
- **Command + Transaction Log** (deposit/withdraw/transfer)
- **Custom Exceptions** and validation
- **In-memory repository** abstraction (easily swappable for DB)
- **Unit tests** and executable scenarios

> Run cells top-to-bottom. Modify the parameters to explore different behaviors.

## Table of Contents
1. [Requirements & Domain](#requirements--domain)
2. [Core Building Blocks (Value Objects, Utilities)](#core-building-blocks)
3. [Account State (State Pattern)](#account-state-state-pattern)
4. [Policies (Strategy Pattern)](#policies-strategy-pattern)
5. [Domain Entities & Transactions](#domain-entities--transactions)
6. [Repository Abstraction](#repository-abstraction)
7. [Bank Service (Application Layer)](#bank-service-application-layer)
8. [Observer Notifications](#observer-notifications)
9. [Scenarios (Demo)](#scenarios-demo)
10. [Unit Tests](#unit-tests)

## Requirements & Domain

**Functional**:
- Create customers and accounts (Checking/Savings)
- Deposit, withdraw, transfer
- Overdraft policy varies by account
- Savings accrue interest; run monthly interest posting
- Freeze/Unfreeze accounts (fraud) and Close accounts
- Record an immutable transaction log

**Non-Functional / Design**:
- Separation of concerns: Entities (core rules) vs Services (workflows)
- Pluggable policies (overdraft/interest) via **Strategy**
- Account status via **State Pattern**
- Testable: in-memory repository + unit tests

**Simplifications**:
- Single currency (`Money` as decimal stored in cents to avoid float errors)
- No concurrency or external I/O; replace repository for production.

### Sketch UML (text)
```
+-------------------+        +-------------------+
| BankService       |        | AccountRepository |
| - repo: ...       |        | + save(a)         |
| + open_account()  |<------>| + by_id(id)       |
| + deposit()       |        | + for_customer(c) |
| + withdraw()      |        +-------------------+
| + transfer()      |
| + post_interest() |        +-------------------+
+-------------------+        | NotificationHub   |
                             | + subscribe(...)  |
+-------------------+        | + publish(evt)    |
| Account (ABC)     |        +-------------------+
| - id, customer... |
| - balance: Money  |<-- Strategy --> OverdraftPolicy
| - state: State    |<-- Strategy --> InterestPolicy
| + deposit()       |
| + withdraw()      |<-- State pattern gates ops
+-------------------+
```

## Core Building Blocks
- `Money` value object storing **cents** (int) with safe arithmetic
- Strongly-typed identifiers (`AccountId`, `CustomerId`, `TxnId`)
- Utilities for time and ULIDs (simple increasing ids for demo)

In [ ]:
from __future__ import annotations
from dataclasses import dataclass
from typing import NewType, Protocol, List, Dict, Callable, Optional, Iterable, Tuple
from datetime import datetime, timezone
import itertools

AccountId = NewType("AccountId", str)
CustomerId = NewType("CustomerId", str)
TxnId = NewType("TxnId", str)

def now_utc() -> datetime:
    return datetime.now(timezone.utc)

# Simple ULID-like increasing ids (demo-friendly)
_counter = itertools.count(1)
def new_id(prefix: str) -> str:
    return f"{prefix}_{next(_counter):06d}"

@dataclass(frozen=True)
class Money:
    cents: int

    @staticmethod
    def from_dollars(amount: float) -> "Money":
        # avoid float issues by rounding to cents
        return Money(int(round(amount * 100)))

    def to_dollars(self) -> float:
        return self.cents / 100.0

    def __add__(self, other: "Money") -> "Money":
        return Money(self.cents + other.cents)

    def __sub__(self, other: "Money") -> "Money":
        return Money(self.cents - other.cents)

    def __lt__(self, other: "Money") -> bool:
        return self.cents < other.cents

    def __le__(self, other: "Money") -> bool:
        return self.cents <= other.cents

    def __repr__(self) -> str:
        return f"${self.to_dollars():.2f}"

ZERO = Money(0)

## Account State (State Pattern)

Account behavior depends on state:
- `Active`: all ops allowed
- `Frozen`: no withdrawals/transfers (deposits allowed)
- `Closed`: no ops

We gate actions through a `State` protocol.

In [ ]:
from abc import ABC, abstractmethod

class AccountState(Protocol):
    name: str
    def can_deposit(self) -> bool: ...
    def can_withdraw(self) -> bool: ...
    def can_transfer(self) -> bool: ...

class ActiveState:
    name = "ACTIVE"
    def can_deposit(self) -> bool: return True
    def can_withdraw(self) -> bool: return True
    def can_transfer(self) -> bool: return True

class FrozenState:
    name = "FROZEN"
    def can_deposit(self) -> bool: return True
    def can_withdraw(self) -> bool: return False
    def can_transfer(self) -> bool: return False

class ClosedState:
    name = "CLOSED"
    def can_deposit(self) -> bool: return False
    def can_withdraw(self) -> bool: return False
    def can_transfer(self) -> bool: return False

## Policies (Strategy Pattern)

**OverdraftPolicy** — controls withdrawals permitted when funds are insufficient.  
**InterestPolicy** — computes monthly interest for Savings.

In [ ]:
class OverdraftPolicy(Protocol):
    def authorize(self, balance: Money, amount: Money) -> bool: ...
    def fee(self) -> Money: ...

class NoOverdraft:
    def authorize(self, balance: Money, amount: Money) -> bool:
        return balance.cents >= amount.cents
    def fee(self) -> Money:
        return ZERO

class FixedLimitOverdraft:
    def __init__(self, limit: Money, fee_amount: Money) -> None:
        self._limit = limit
        self._fee = fee_amount
    def authorize(self, balance: Money, amount: Money) -> bool:
        return (balance.cents - amount.cents) >= -self._limit.cents
    def fee(self) -> Money:
        return self._fee

class InterestPolicy(Protocol):
    def monthly_interest(self, balance: Money) -> Money: ...

class TieredInterest:
    """Example: 1% for first $5k, then 2% beyond, prorated monthly."""
    def __init__(self, annual_low=0.01, annual_high=0.02, threshold=Money.from_dollars(5000)):
        self.annual_low = annual_low
        self.annual_high = annual_high
        self.threshold = threshold
    def monthly_interest(self, balance: Money) -> Money:
        if balance <= self.threshold:
            return Money(int(round(balance.cents * (self.annual_low/12))))
        low_part = self.threshold.cents * (self.annual_low/12)
        high_part = (balance.cents - self.threshold.cents) * (self.annual_high/12)
        return Money(int(round(low_part + high_part)))

## Domain Entities & Transactions

- `Customer`
- `Account` (abstract) ⇒ `CheckingAccount` / `SavingsAccount`
- `Transaction` (immutable log of operations)
- Domain exceptions

In [ ]:
from dataclasses import dataclass, field

# Exceptions
class DomainError(Exception): ...
class AccountStateError(DomainError): ...
class InsufficientFunds(DomainError): ...
class AccountClosed(DomainError): ...
class AccountFrozen(DomainError): ...

@dataclass
class Customer:
    id: CustomerId
    name: str
    email: str

@dataclass(frozen=True)
class Transaction:
    id: TxnId
    timestamp: datetime
    kind: str            # 'DEPOSIT' | 'WITHDRAW' | 'TRANSFER' | 'FEE' | 'INTEREST'
    amount: Money
    src_account: Optional[AccountId] = None
    dst_account: Optional[AccountId] = None
    memo: str = ""

class Account(ABC):
    def __init__(self, id: AccountId, customer_id: CustomerId, overdraft: OverdraftPolicy) -> None:
        self.id = id
        self.customer_id = customer_id
        self._balance: Money = ZERO
        self._state: AccountState = ActiveState()
        self._overdraft = overdraft
        self._txns: List[Transaction] = []

    @property
    def balance(self) -> Money:
        return self._balance

    @property
    def state(self) -> str:
        return self._state.name

    def freeze(self) -> None:
        self._state = FrozenState()

    def unfreeze(self) -> None:
        self._state = ActiveState()

    def close(self) -> None:
        self._state = ClosedState()

    def _append_txn(self, txn: Transaction) -> None:
        self._txns.append(txn)

    def transactions(self) -> Iterable[Transaction]:
        return tuple(self._txns)

    # Domain operations
    def deposit(self, amount: Money, memo: str = "") -> Transaction:
        if not self._state.can_deposit(): raise AccountClosed("Deposits not allowed in current state")
        if amount.cents <= 0: raise DomainError("Deposit must be positive")
        self._balance = self._balance + amount
        txn = Transaction(TxnId(new_id("txn")), now_utc(), "DEPOSIT", amount, dst_account=self.id, memo=memo)
        self._append_txn(txn)
        return txn

    def withdraw(self, amount: Money, memo: str = "") -> List[Transaction]:
        if not self._state.can_withdraw():
            raise AccountFrozen("Withdrawals not allowed while account is not Active")
        if amount.cents <= 0: raise DomainError("Withdraw must be positive")
        txns: List[Transaction] = []
        if not self._overdraft.authorize(self._balance, amount):
            raise InsufficientFunds("Overdraft policy denied withdrawal")
        # main withdrawal
        self._balance = self._balance - amount
        txns.append(Transaction(TxnId(new_id("txn")), now_utc(), "WITHDRAW", amount, src_account=self.id, memo=memo))
        # fee if overdraft used
        fee = self._overdraft.fee()
        if fee.cents > 0 and self._balance.cents < 0:
            self._balance = self._balance - fee
            txns.append(Transaction(TxnId(new_id("txn")), now_utc(), "FEE", fee, src_account=self.id, memo="overdraft fee"))
        for t in txns: self._append_txn(t)
        return txns

    @abstractmethod
    def account_type(self) -> str: ...

class CheckingAccount(Account):
    def account_type(self) -> str: return "CHECKING"

class SavingsAccount(Account):
    def __init__(self, id: AccountId, customer_id: CustomerId, overdraft: OverdraftPolicy, interest: InterestPolicy) -> None:
        super().__init__(id, customer_id, overdraft)
        self._interest = interest

    def post_monthly_interest(self) -> Transaction:
        if isinstance(self._state, ClosedState):
            raise AccountClosed("Interest posting not allowed on closed account")
        interest = self._interest.monthly_interest(self.balance)
        if interest.cents <= 0:
            # still record zero? we'll skip to reduce noise
            return Transaction(TxnId(new_id("txn")), now_utc(), "INTEREST", ZERO, dst_account=self.id, memo="no interest (low balance)")
        self._balance = self._balance + interest
        txn = Transaction(TxnId(new_id("txn")), now_utc(), "INTEREST", interest, dst_account=self.id, memo="monthly interest")
        self._append_txn(txn)
        return txn

    def account_type(self) -> str: return "SAVINGS"

## Repository Abstraction

Allows swapping the persistence mechanism without changing domain logic.

In [ ]:
class AccountRepository(Protocol):
    def save(self, account: Account) -> None: ...
    def by_id(self, id: AccountId) -> Optional[Account]: ...
    def for_customer(self, customer_id: CustomerId) -> List[Account]: ...

class InMemoryAccountRepository:
    def __init__(self):
        self._store: Dict[str, Account] = {}
    def save(self, account: Account) -> None:
        self._store[str(account.id)] = account
    def by_id(self, id: AccountId) -> Optional[Account]:
        return self._store.get(str(id))
    def for_customer(self, customer_id: CustomerId) -> List[Account]:
        return [a for a in self._store.values() if a.customer_id == customer_id]

## Observer Notifications

A simple hub so services can publish events (e.g., low balance, overdraft).

In [ ]:
@dataclass(frozen=True)
class Event:
    name: str
    data: dict

class NotificationHub:
    def __init__(self):
        self._subs: Dict[str, List[Callable[[Event], None]]] = {}
    def subscribe(self, event_name: str, handler: Callable[[Event], None]) -> None:
        self._subs.setdefault(event_name, []).append(handler)
    def publish(self, event_name: str, data: dict) -> None:
        for h in self._subs.get(event_name, []):
            h(Event(event_name, data))

## Bank Service (Application Layer)

Coordinates repositories, policies, and domain entities to implement **use cases**.

In [ ]:
class BankService:
    def __init__(self, repo: AccountRepository, notifier: NotificationHub):
        self.repo = repo
        self.notifier = notifier

    def open_checking(self, customer_id: CustomerId, overdraft: OverdraftPolicy) -> CheckingAccount:
        acc = CheckingAccount(AccountId(new_id("acct")), customer_id, overdraft)
        self.repo.save(acc)
        return acc

    def open_savings(self, customer_id: CustomerId, overdraft: OverdraftPolicy, interest: InterestPolicy) -> SavingsAccount:
        acc = SavingsAccount(AccountId(new_id("acct")), customer_id, overdraft, interest)
        self.repo.save(acc)
        return acc

    def deposit(self, account_id: AccountId, amount: Money, memo: str = "") -> Transaction:
        acc = self._get(account_id)
        txn = acc.deposit(amount, memo)
        self.repo.save(acc)
        return txn

    def withdraw(self, account_id: AccountId, amount: Money, memo: str = "") -> List[Transaction]:
        acc = self._get(account_id)
        txns = acc.withdraw(amount, memo)
        if acc.balance.cents < 0:
            self.notifier.publish("overdraft", {"account": str(acc.id), "balance": acc.balance.to_dollars()})
        if acc.balance.cents < Money.from_dollars(50).cents:
            self.notifier.publish("low_balance", {"account": str(acc.id), "balance": acc.balance.to_dollars()})
        self.repo.save(acc)
        return txns

    def transfer(self, src_id: AccountId, dst_id: AccountId, amount: Money, memo: str = "") -> List[Transaction]:
        if src_id == dst_id: raise DomainError("Cannot transfer to same account")
        src = self._get(src_id)
        dst = self._get(dst_id)
        if not src._state.can_transfer() or not dst._state.can_deposit():
            raise AccountStateError("Invalid state for transfer")
        wtxns = src.withdraw(amount, memo or f"transfer to {dst_id}")
        dtxn = dst.deposit(amount, memo or f"transfer from {src_id}")
        self.repo.save(src); self.repo.save(dst)
        return [*wtxns, dtxn]

    def post_interest_all(self, customer_id: CustomerId) -> List[Transaction]:
        txns: List[Transaction] = []
        for acc in self.repo.for_customer(customer_id):
            if isinstance(acc, SavingsAccount):
                t = acc.post_monthly_interest()
                txns.append(t)
                self.repo.save(acc)
        return txns

    def freeze(self, account_id: AccountId) -> None:
        acc = self._get(account_id); acc.freeze(); self.repo.save(acc)
    def unfreeze(self, account_id: AccountId) -> None:
        acc = self._get(account_id); acc.unfreeze(); self.repo.save(acc)
    def close(self, account_id: AccountId) -> None:
        acc = self._get(account_id); acc.close(); self.repo.save(acc)

    def _get(self, account_id: AccountId) -> Account:
        acc = self.repo.by_id(account_id)
        if not acc: raise DomainError("Account not found")
        return acc

## Scenarios (Demo)

Let's run a realistic workflow:
1. Open Checking (with overdraft limit) and Savings (tiered interest)
2. Deposit, withdraw, and transfer
3. Trigger overdraft and low-balance notifications
4. Post monthly interest

In [ ]:
# Setup service
repo = InMemoryAccountRepository()
hub = NotificationHub()

# Subscribe demo handlers
events = []
hub.subscribe("overdraft", lambda e: events.append((e.name, e.data)))
hub.subscribe("low_balance", lambda e: events.append((e.name, e.data)))

svc = BankService(repo, hub)

# Open accounts
cust_id = CustomerId("cust_001")
chk = svc.open_checking(cust_id, overdraft=FixedLimitOverdraft(Money.from_dollars(200), Money.from_dollars(35)))
sav = svc.open_savings(cust_id, overdraft=NoOverdraft(), interest=TieredInterest())

print("Checking:", chk.id, chk.account_type(), chk.balance)
print("Savings :", sav.id, sav.account_type(), sav.balance)

# Deposits
svc.deposit(chk.id, Money.from_dollars(300), "initial deposit")
svc.deposit(sav.id, Money.from_dollars(6000), "seed savings")
print("Balances after deposit:", repo.by_id(chk.id).balance, repo.by_id(sav.id).balance)

# Withdraw and overdraft scenario
svc.withdraw(chk.id, Money.from_dollars(450), "rent payment")  # will use overdraft and fee
print("Checking after rent:", repo.by_id(chk.id).balance)

# Transfer from savings to checking to cover overdraft
svc.transfer(sav.id, chk.id, Money.from_dollars(500), "cover overdraft")
print("Balances after transfer:", repo.by_id(chk.id).balance, repo.by_id(sav.id).balance)

# Post interest for savings
interest_txns = svc.post_interest_all(cust_id)
print("Interest posted:", [t.amount for t in interest_txns if t.amount.cents>0])
print("Savings after interest:", repo.by_id(sav.id).balance)

# Show events
print("Events:", events[:5])

In [ ]:
# Inspect transaction log for checking
chk_reloaded = repo.by_id(chk.id)
for t in chk_reloaded.transactions():
    print(t.timestamp.isoformat(), t.kind, t.amount, t.memo)

## Unit Tests

A few tests to verify policies and state transitions.

In [ ]:
def run_tests():
    repo = InMemoryAccountRepository()
    hub = NotificationHub()
    svc = BankService(repo, hub)
    cust = CustomerId("c1")
    chk = svc.open_checking(cust, FixedLimitOverdraft(Money.from_dollars(100), Money.from_dollars(25)))
    svc.deposit(chk.id, Money.from_dollars(50))
    assert repo.by_id(chk.id).balance == Money.from_dollars(50)

    # Overdraft allowed up to -$100
    svc.withdraw(chk.id, Money.from_dollars(120))
    bal = repo.by_id(chk.id).balance
    assert bal.cents < 0  # in overdraft

    # Freeze prevents withdraw/transfer
    svc.freeze(chk.id)
    try:
        svc.withdraw(chk.id, Money.from_dollars(10))
        assert False, "withdraw should fail when frozen"
    except AccountFrozen:
        pass
    svc.unfreeze(chk.id)

    # Savings interest
    sav = svc.open_savings(cust, NoOverdraft(), TieredInterest())
    svc.deposit(sav.id, Money.from_dollars(6000))
    txns = svc.post_interest_all(cust)
    assert any(t.kind == "INTEREST" and t.amount.cents > 0 for t in txns)

    print("All tests passed ✅")


run_tests()